In [7]:
import tempfile
import urllib.request
train_file = "datasets/thermostat/sample-training-data.csv"
test_file = "datasets/thermostat/test-data.csv"

In [8]:
import pandas as pd
COLUMNS = ["month", "day", "hour", "min", "pirstatus",
           "isDay", "extTemp", "extHumidity", "loungeTemp", "loungeHumidity",
           "state", "temperature", "label"]
df_train = pd.read_csv(train_file, names=COLUMNS, skipinitialspace=True)
df_test = pd.read_csv(test_file, names=COLUMNS, skipinitialspace=True)

In [9]:
CATEGORICAL_COLUMNS = []
CONTINUOUS_COLUMNS = ["month","day", "hour", "min", "pirstatus",
                      "isDay", "extTemp", "extHumidity", "loungeTemp", "loungeHumidity"
           ]
LABEL_COLUMN="label"

In [10]:
df_train[LABEL_COLUMN] = df_train["state"]
df_test[LABEL_COLUMN] = df_test["state"]
print(df_test)

   month  day  hour  min  pirstatus  isDay  extTemp  extHumidity  loungeTemp  \
0     12   12     0    0          1      0       19           52        19.5   
1     12   12     9    0          0      1       19           52        18.0   
2     12   12    10    0          0      1       21           52        19.0   
3     12   12    14    0          0      1       24           52        22.0   
4     12   12    16    0          1      1       23           52        22.0   

   loungeHumidity  state  temperature  label  
0              53      0           26      0  
1              53      0           26      0  
2              53      0           26      0  
3              53      1           24      1  
4              53      1           24      1  


In [11]:
import tensorflow as tf

def input_fn(df):
  # Creates a dictionary mapping from each continuous feature column name (k) to
  # the values of that column stored in a constant Tensor.
  continuous_cols = {k: tf.constant(df[k].values)
                     for k in CONTINUOUS_COLUMNS}
  # Creates a dictionary mapping from each categorical feature column name (k)
  # to the values of that column stored in a tf.SparseTensor.
  categorical_cols = {k: tf.SparseTensor(
      indices=[[i, 0] for i in range(df[k].size)],
      values=df[k].values.astype(str),
      dense_shape=[df[k].size, 1])
                      for k in CATEGORICAL_COLUMNS}
  # Merges the two dictionaries into one.
  feature_cols = dict()
  feature_cols.update(continuous_cols.copy())
  feature_cols.update(categorical_cols.copy())
  #feature_cols = dict(continuous_cols.items() + categorical_cols.items())
  # Converts the label column into a constant Tensor.
  label = tf.constant(df[LABEL_COLUMN].values)
  # Returns the feature columns and the label.
  return feature_cols, label

def train_input_fn():
  return input_fn(df_train)

def eval_input_fn():
  return input_fn(df_test)



In [12]:
month = tf.contrib.layers.real_valued_column("month")
day = tf.contrib.layers.real_valued_column("day")
hour = tf.contrib.layers.real_valued_column("hour")
minute = tf.contrib.layers.real_valued_column("min")
pirstatus = tf.contrib.layers.real_valued_column("pirstatus")
isDay = tf.contrib.layers.real_valued_column("isDay")
extTemp = tf.contrib.layers.real_valued_column("extTemp")
extHumidity = tf.contrib.layers.real_valued_column("extHumidity")
loungeTemp = tf.contrib.layers.real_valued_column("loungeTemp")
loungeHumidity = tf.contrib.layers.real_valued_column("loungeHumidity")

In [13]:
model_dir = tempfile.mkdtemp()
m = tf.contrib.learn.LinearClassifier(feature_columns=[
  month, day, hour, minute, pirstatus, isDay,
  extTemp, extHumidity, loungeTemp, loungeHumidity],
  optimizer=tf.train.FtrlOptimizer(
    learning_rate=0.1,
    l1_regularization_strength=1.0,
    l2_regularization_strength=1.0),
  model_dir=model_dir)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_save_checkpoints_secs': 600, '_num_worker_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025184738B38>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_environment': 'local', '_task_type': None, '_model_dir': 'C:\\Users\\faisal.t\\AppData\\Local\\Temp\\tmpueabmsxd', '_session_config': None, '_is_chief': True, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_num_ps_replicas': 0, '_save_summary_steps': 100}


In [14]:
m.fit(input_fn=train_input_fn, steps=500)

Instructions for updating:
Please switch to tf.train.get_global_step
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\faisal.t\AppData\Local\Temp\tmpueabmsxd\model.ckpt.
INFO:tensorflow:loss = 0.693147, step = 1
INFO:tensorflow:global_step/sec: 730.026
INFO:tensorflow:loss = 0.2893, step = 101 (0.139 sec)
INFO:tensorflow:global_step/sec: 1063.84
INFO:tensorflow:loss = 0.219083, step = 201 (0.094 sec)
INFO:tensorflow:global_step/sec: 1063.84
INFO:tensorflow:loss = 0.182496, step = 301 (0.095 sec)
INFO:tensorflow:global_step/sec: 1041.63
INFO:tensorflow:loss = 0.159112, step = 401 (0.095 sec)
INFO:tensorflow:Saving checkpoints for 500 into C:\Users\faisal.t\AppData\Local\Temp\tmpueabmsxd\model.ckpt.
INFO:tensorflow:Loss for final step: 0.142792.


LinearClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x000002518473A908>, 'gradient_clip_norm': None, 'joint_weights': False, 'feature_columns': [_RealValuedColumn(column_name='month', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='day', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='hour', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='min', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='pirstatus', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='isDay', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='extTemp', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='e

In [15]:
results = m.evaluate(input_fn=eval_input_fn, steps=1)
print("printin results")
for key in sorted(results):
    print("%s: %s" % (key, results[key]))

ValueError: Features are incompatible with given information. Given features: {'month': <tf.Tensor 'Const:0' shape=(5,) dtype=int64>, 'pirstatus': <tf.Tensor 'Const_4:0' shape=(5,) dtype=int64>, 'isDay': <tf.Tensor 'Const_5:0' shape=(5,) dtype=int64>, 'day': <tf.Tensor 'Const_1:0' shape=(5,) dtype=int64>, 'hour': <tf.Tensor 'Const_2:0' shape=(5,) dtype=int64>, 'min': <tf.Tensor 'Const_3:0' shape=(5,) dtype=int64>, 'extHumidity': <tf.Tensor 'Const_7:0' shape=(5,) dtype=int64>, 'loungeHumidity': <tf.Tensor 'Const_9:0' shape=(5,) dtype=int64>, 'loungeTemp': <tf.Tensor 'Const_8:0' shape=(5,) dtype=float64>, 'extTemp': <tf.Tensor 'Const_6:0' shape=(5,) dtype=int64>}, required signatures: {'day': TensorSignature(dtype=tf.int64, shape=TensorShape([Dimension(72)]), is_sparse=False), 'hour': TensorSignature(dtype=tf.int64, shape=TensorShape([Dimension(72)]), is_sparse=False), 'min': TensorSignature(dtype=tf.int64, shape=TensorShape([Dimension(72)]), is_sparse=False), 'extHumidity': TensorSignat

In [20]:
def predict_input_fn():
  test_data = {
    "month":[12],
    "day":[12],
    "hour":[22],
    "min":[0],
    "pirstatus":[0],
    "isDay":[1],
    "extTemp":[35],
    "extHumidity":[20],
    "loungeTemp":[12],
    "loungeHumidity":[30],
  }
  
  continuous_cols = {k: tf.constant(test_data[k])
                     for k in test_data}
  return continuous_cols

In [21]:
predictions = list(m.predict(input_fn=predict_input_fn, as_iterable=True))
print('Predictions: {}'.format(str(predictions)))

INFO:tensorflow:Restoring parameters from C:\Users\faisal.t\AppData\Local\Temp\tmpueabmsxd\model.ckpt-500
Predictions: [1]
